LIMPIEZA Y TRANSFORMACIÓN DE DATOS

Por hacer:
* Eliminar columnas que no se van a usar y documentar por qué
* Verificar tipos de datos en el df_gen 
* Buscar datos nulos en los df y mirar que técnica usar para reemplazarlos
* Tratar los df_in_time y df_out_time (opcional, no es calificable)
* Para calcular nuevas variables, filtros, hacer consultas y unir los df, se debe usar SQL

In [1]:
# Cargar librerías
import pandas as pd  # Manejo de dataframes
import sqlite3 as sql  # Manejo de base de datos

In [191]:
# Cargar data
df_emp = pd.read_csv("data/employee_survey_data.csv")
df_gen = pd.read_csv("data/general_data.csv", sep=";")
df_in_time = pd.read_csv("data/in_time.csv")
df_manager = pd.read_csv("data/manager_survey_data.csv")
df_out_time = pd.read_csv("data/out_time.csv")
df_retirement = pd.read_csv("data/retirement_info.csv", sep=";")

In [192]:
# Renombrar columnas
df_in_time = df_in_time.rename(columns={"Unnamed: 0": "EmployeeID"})
df_out_time = df_out_time.rename(columns={"Unnamed: 0": "EmployeeID"})

# Establecer índices
df_emp = df_emp.set_index("EmployeeID")
df_gen = df_gen.set_index("EmployeeID")
df_in_time = df_in_time.set_index("EmployeeID")
df_manager = df_manager.set_index("EmployeeID")
df_out_time = df_out_time.set_index("EmployeeID")
df_retirement = df_retirement.set_index("EmployeeID")

# Convertir a formato fecha
df_in_time = df_in_time.apply(pd.to_datetime)
df_out_time = df_out_time.apply(pd.to_datetime)

df_retirement["retirementDate"] = pd.to_datetime(
    df_retirement["retirementDate"], format="%d/%m/%Y"
)

# Convertir columnas a formato float
df_manager["JobInvolvement"] = df_manager["JobInvolvement"].astype(float)
df_manager["PerformanceRating"] = df_manager["PerformanceRating"].astype(float)

In [208]:
# Mostrar todas las columnas
pd.set_option("display.max_columns", None)
df_emp.head()

,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
EmployeeID,,,
1,3.0,4.0,2.0
2,3.0,2.0,4.0
3,2.0,2.0,1.0
4,4.0,4.0,3.0
5,4.0,1.0,3.0
